In [1]:
import torch
from transformers import BertForMaskedLM, BertTokenizer, AutoTokenizer, AutoModel, pipeline
from datasets import Dataset, load_dataset
import re
import numpy as np
import os
import requests
import pandas as pd
from tqdm import tqdm

2024-01-26 15:05:57.779863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Get model and tokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [3]:
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert_bfd")

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
tqdm.pandas()

In [6]:
def get_perplexity(sequence):
    sequence = " ".join(re.sub(r"[UZOB]", "X", sequence))
    tokenized_text = tokenizer.tokenize(sequence)
    
    
    # Track log likelihoods
    log_likelihoods = []

    for masked_index in range(1, len(tokenized_text) - 1):  # Avoid masking [CLS] and [SEP]
        # Create a copy of the token list and mask one token
        masked_tokens = tokenized_text.copy()
        masked_tokens[masked_index] = '[MASK]'
        masked_text = ' '.join(masked_tokens).replace(' ##', '')

        # Tokenize masked sentence
        inputs = tokenizer(masked_text, return_tensors="pt")

        # Move input tensors to the GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get model predictions for the masked token
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = outputs.logits

        # Get the softmax probabilities
        softmax = torch.nn.Softmax(dim=-1)
        mask_token_logits = predictions[0, masked_index]
        mask_token_probs = softmax(mask_token_logits)

        # Find the true token's probability
        true_token_id = inputs['input_ids'][0, masked_index]
        true_token_prob = mask_token_probs[true_token_id]

        # Calculate the negative log likelihood
        log_likelihood = torch.log(true_token_prob)
        log_likelihoods.append(log_likelihood)

    # Calculate the average negative log likelihood
    average_neg_log_likelihood = -torch.stack(log_likelihoods).mean().item()

    # Estimate the average perplexity per token
    average_perplexity = torch.exp(torch.tensor(average_neg_log_likelihood))
    return average_perplexity.item()

In [12]:
dataset = load_dataset('EvaKlimentova/Diffusion-all_knots')['train']
df = pd.DataFrame(dataset)[['ID', 'Sequence', 'Tool']]
df = df[df['Tool'] != 'Real'].reset_index(drop=True)

Using custom data configuration EvaKlimentova--Diffusion-all_knots-f1f7b10e1861ce17
Found cached dataset parquet (/home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--Diffusion-all_knots-f1f7b10e1861ce17/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
df

,ID,Sequence,Tool
0,RF_4-426_0,SLEEERKKKFIEDFRALMDVLLDRILEEIKKLCEKENKDVIIMVFV...,RFdiffusion
1,RF_4-55_0,AAIQAELEAEVARLAAAMPAVMARVAEEAKKAASLSFLELLAVLSG...,RFdiffusion
2,RF_5-600_0,KTVVLKDLPKEEMKKKLKEAAKAGDKIRIVITPENAEEVLEVIKEL...,RFdiffusion
3,RF_5-395_0,MKVLISRATIRVLSVDDEHELRFVYNEETGNLEVITVTLSEGVAYI...,RFdiffusion
4,RF_10_61_1,KKIFELTLVVSTEELALEILEELSKSCELTLVATPEGFTLLILCAL...,RFdiffusion
...,...,...,...
4995,RF_5-1318_0,SMEEVLEATVELETEAELEDLLRLIALVVALQPDARVLVAAEDGVL...,RFdiffusion
4996,RF_8_256_1,MKEKKKAVLVDITFIPASRITPETLEKMKELQAKMVAALKAGDVET...,RFdiffusion
4997,RF_1-441_0,SEKAEQLQAILDEYIEMLEKELKKKYKGLELKSIKFPLIVYDGEKD...,RFdiffusion
4998,RF_5-661_0,GLWEEVKQLVKEMKVDKEKGTLTIELVVETKDGTVLRAKITVTLPT...,RFdiffusion


In [ ]:
perplexity = df['Sequence'].progress_apply(get_perplexity)

  0%|          | 18/5000 [02:28<14:20:51, 10.37s/it]

In [ ]:
df['Perplexity'] = perplexity

In [20]:
df

,ID,Sequence,Tool,Perplexity
0,RF_4-426_0,SLEEERKKKFIEDFRALMDVLLDRILEEIKKLCEKENKDVIIMVFV...,RFdiffusion,1.397958
1,RF_4-55_0,AAIQAELEAEVARLAAAMPAVMARVAEEAKKAASLSFLELLAVLSG...,RFdiffusion,1.452535
2,RF_5-600_0,KTVVLKDLPKEEMKKKLKEAAKAGDKIRIVITPENAEEVLEVIKEL...,RFdiffusion,1.417952
3,RF_5-395_0,MKVLISRATIRVLSVDDEHELRFVYNEETGNLEVITVTLSEGVAYI...,RFdiffusion,1.398252
4,RF_10_61_1,KKIFELTLVVSTEELALEILEELSKSCELTLVATPEGFTLLILCAL...,RFdiffusion,1.421470
...,...,...,...,...
4995,RF_5-1318_0,SMEEVLEATVELETEAELEDLLRLIALVVALQPDARVLVAAEDGVL...,RFdiffusion,1.399607
4996,RF_8_256_1,MKEKKKAVLVDITFIPASRITPETLEKMKELQAKMVAALKAGDVET...,RFdiffusion,1.416514
4997,RF_1-441_0,SEKAEQLQAILDEYIEMLEKELKKKYKGLELKSIKFPLIVYDGEKD...,RFdiffusion,1.414330
4998,RF_5-661_0,GLWEEVKQLVKEMKVDKEKGTLTIELVVETKDGTVLRAKITVTLPT...,RFdiffusion,1.459369


In [ ]:
df.to_csv('Artificial_proteins_perplexity.csv', index=False)

In [21]:
df.describe(include='all')

,ID,Sequence,Tool,Perplexity
count,5000,5000,5000,5000.000000
unique,5000,5000,1,NaN
top,RF_4-426_0,SLEEERKKKFIEDFRALMDVLLDRILEEIKKLCEKENKDVIIMVFV...,RFdiffusion,NaN
freq,1,1,5000,NaN
mean,NaN,NaN,NaN,1.403765
std,NaN,NaN,NaN,0.048862
min,NaN,NaN,NaN,1.111967
25%,NaN,NaN,NaN,1.377371
50%,NaN,NaN,NaN,1.410344
75%,NaN,NaN,NaN,1.437371


---------------